# PostgreSQL
> [Main Table of Contents](../README.md)

## In This Notebook
- Useful commands
- Install PostgreSQL on Rasp Pi OS
- Transfer data from Google cloud postgres to postgres on Raspberry Pi 4
    - Subsequent logins
    - First time setup of gcloud and cloud-sql-proxy

## Useful commands
- Check if PostgreSQL is running
    ```
    systemctl status postgresql
    ```
- Start PostgreSQL
    ```
    systemctl start psotgresql
    ```
- Stop PostgreSQL
    ```
    systemctl stop psotgresql
    ```
- Enter Postgres via CLI
    - Change to postgres super user
        ```
        sudo su postgres
        ```
    - Enter postgres
        ```
        psql   -or-
        psql <databaseName>
        ```
        - List all databases
            ```
            \l
            ```
        - List all database users
            ```
            \du
            ```
        - Connect to db
            ```
            \c <databaseName>
            ```
            - List all tables in connected db
                ```
                \dt
                ```
        - Change ownership of a table
            ```
            ALTER TABLE <tableName> OWNER TO <postgres_username>
            e.g. ALTER TABLE tickers OWNER TO river
            ```

## Install PostgreSQL on Rasp Pi OS
> Reference: [Install postgresql on raspberry pi](https://singleboardblog.com/install-postgresql-on-raspberry-pi/)  
> Reference: [Raspberry Pi Docs](https://www.raspberrypi.com/documentation/computers/remote-access.html#introduction-to-remote-access)
- PostgreSQL is already installed on Rasp pi os
    1. [Download Raspberry Pi OS using Rasp Pi Imager](https://www.raspberrypi.com/software/)
    2. Insert micro SD card to USB of laptop
    3. Run Imager
    4. Choose the 64 bit Raspberry Pi OS for Rasp Pi 4 (not 32 bit) 
    5. Remove micro SD from laptop and insert into Rasp Pi 4
- Open terminal
    - Update the apt repository
        ```
        sudo apt update && sudo apt upgrade -y
        ```
    - Install PostgreSQL on Rasp Pi
        ```
        sudo apt install postgresql
        ```
    - Check if PostgreSQL is running (should automatically run)
        ```
        sudo systemctl status postgresql
        ```
    - Switch to the "postgres" user (only user that is recognized) to establish initial connection
        ```
        sudo su postgres
        ```
    - Create a new user (not super user) & user password for PostgreSQL
        ```
        createuser <username> -P --interactive
        
        e.g. createuser meeemeee -P --interactive
        ```
        - Create password for this new user (e.g. bestEverPassword)
        - Answer n for superuser
        - Answer y to allow user to create databases
        - Answer y to allow user to create more new roles
    - Connect to PostgreSQL CLI
        ```
        psql -d postgres -U <username> -W

        e.g. psql -d postgres -U meeemeee -W
        ```
    - Create a new database
        ```
        CREATE DATABASE mytestdb
        ```
    - Give access to user for databases. Make changes in `pg_hba.conf` file
        - Open file
            ```
            nano /etc/postgresql/13/main/pg_hba.conf
            ```
        - Insert one of the two new line options below
            ```
            # Limited access to mytestdb database by one meeemeee username only
            host <database> <user> 0.0.0.0/0 md5
            e.g. host mytestdb meeemeee 0.0.0.0/0 md5

            # Give access to all user to all databases
            host all all 0.0.0.0/0 md5
            ```
    - Make database listen for remote conneections. Make changes in `postgresql.conf` file
        > By default, PostgreSQL server is listening only for the local connections
        - Open file
            ```
            nano /etc/postgresql/13/main/postgresql.conf
            ```
        - Change `listen_addresses` line
            ```
            # Change listen_addresses = 'localhost' to 
            listen_addresses = '*'
            ```
    - Restart PostgreSQL
        ```
        systemctl restart postgresql
        ```
    - Get host ip address
        ```
        hostname -I
        ```
    - Connect to db via CLI
        ```
        psql -h <host ip> -p 5432 -d <database> -U <username> -W

        e.g. psql -h 192.168.1.30 -p 5432 -d mytestdb -U meeemeee -W
        ```
    - Connect to db via python psycopg2
        ```
        from psycopg2 import connect

        conn = connect(
            host=<host ip address e.g. 10.0.0.151>,
            port='5432'.
            database='mytestdb',
            user='meeemeee',
            password='bestEverPassword'
        )
        ```

## Transfer data from Google cloud postgres to postgres on Raspberry Pi 4
> Google PostgreSQL uses managed endpoints and doesn't give me a virtual machine to SSH into


### Subsequent logins
WSL Linux terminal:
1. `gcloud init`
2. `<pwd>/cloud-sql-proxy GOOGLE_INSTANCE_CONNECTION_NAME` e.g. `/home/huej/cloud-sql-proxy my_instance_name-123566:us-central70:abc`

### First time setup of gcloud and cloud-sql-proxy
1. On my WSL linux terminal: Connect [CloudSQL instance with proxy](https://cloud.google.com/sql/docs/postgres/connect-instance-auth-proxy) and run all the commands to get this setup
    - If `<pwd>/cloud-sql-proxy INSTANCE_CONNECTION_NAME` doesn't work due to credentials, follow these steps for [ADC](https://cloud.google.com/docs/authentication/provide-credentials-adc#how-to) for local development environment
2. Now my google cloud Postgres is available on a localhost e.g. 127.0.0.1:5432  
3. Export data into SQL file
    ```python
    # Open new WSL terminal
    # This will create a SQL file named dump in my WSL
    pg_dump -U <username> -h <host> -p <port> -d <database_name> -f dump.sql  # creds of GCP not rasp pi
    
    e.g. pg_dump -U postgres -h 127.0.0.1 -p 5432 -d chief -f dump.sql
    ```
4. If need raspberry pi ip address
    ```python
    # Get raspi_ipaddress
    ping raspberypi.local
    ```
5. Securely transfer the file from local WSL to Rasp Pi 4 (which I setup SSH capabilities)
    ```python
    # Open new WSL terminal
    scp ./dump.sql <raspi_username>@<raspi_ipaddress>:/path/to/destination
    ```
6. Open new WSL terminal and SSH into rasp pi
    ```python
    # SSH into rasp pi
    ssh <raspi_username>@<raspi_ipaddress>
    ```
7. Import SQL file
    ```python
    # If non "postgres" username name doesn't work, just use "postgres" as username and see "Useful commands" section above in changing ownership of tables in a db
    psql -U <username> -d <database_name> -f dump.sql
    ```